# Collette Report (2021 & 2022)

In [ ]:
import numpy as np
import pandas as pd 
import pandas_profiling
from datetime import datetime
from matplotlib import pyplot as plt

## Set Variables

In [ ]:
report_ID_2022 = 2022054
today = datetime(year=2022, month=1, day=7)
current_raw_file = 'Reports/collette_raw_data_01-07-22.csv'
previous_report_file = 'Reports/collette_data_12-31-21.csv'
new_report_file = 'Reports/collette_data_01-07-22.csv'

## OpCode Dictionary

In [ ]:
collette_trips = {
    "Alaska and The Yukon featuring the Yukon, Fairbanks and Denali" : "ColletteAY",
    "Alaska Discovery Land & Cruise featuring a 7-night Princess Cruise" : "ColletteADLC",
    "Alaska: America's Last Frontier" : "ColletteAALF",
    "Alaska's Northern Lights" : "ColletteANL",
    "Alaska's Northern Lights featuring the Iditarod Race" : "ColletteANLI",
    "Albuquerque Balloon Fiesta" : "ColletteABF",
    "America's Cowboy Country" : "ColletteACC",
    "America's Music Cities featuring New Orleans Jazz Fest, Memphis & Nashville" : "ColletteAMCJF",
    "America's Music Cities featuring Nashville, Memphis & New Orleans Jazz Fest" : "ColletteAMCJF",
    "America's Music Cities featuring Nashville, Memphis & New Orleans" : "ColletteAMC",
    "America's Music Cities featuring New Orleans, Memphis & Nashville" : "ColletteAMC",
    "America's Music Cities Holiday featuring New Orleans, Memphis & Nashville" : "ColletteAMCH",
    "America's National Parks & Denver" : "ColletteANPD",
    "Autumn in Vermont featuring Lake Champlain and the Adirondacks" : "ColletteAV",
    "Bluegrass Country & the Smoky Mountains featuring Louisville, Gatlinburg & Asheville" : "ColletteBCSM",
    "Bluegrass Country & the Smoky Mountains featuring Louisville, Lexington, Gatlinburg & Asheville" : "ColletteBCSM",
    "California Dreamin': Monterey, Yosemite & Napa" : "ColletteCD",
    "Canadian Rockies & Glacier National Park" : "ColletteCRGNP",
    "Canadian Rockies & Glacier National Park featuring the Calgary Stampede" : "ColletteCRGNPCS",
    "Canadian Rockies by Train" : "ColletteCRT",
    "Canadian Rockies by Train featuring the Calgary Stampede" : "ColletteCRTCS",
    "Canadian Rockies featuring Rocky Mountaineer" : "ColletteCRRM",
    "Canadian Rockies featuring Rocky Mountaineer and Calgary Stampede" : "ColletteCRRMCS",
    "Canyon Country featuring Arizona & Utah" : "ColletteCC",
    "Charming French Canada featuring Montreal, Quebec City, Charlevoix and Montebello" : "ColletteCFC",
    "Colors of New England featuring Coastal Maine" : "ColletteCNE",
    "Colours of New England featuring Coastal Maine" : "ColletteCNE",
    "Colors of New England featuring Portland, Maine" : "ColletteCNEP",
    "Colours of New England featuring Portland, Maine" : "ColletteCNEP",
    "Costa Rica: A World of Nature featuring Tortuguero National Park, Arenal Volcano & Manuel Antonio National Park" : "ColletteCR",
    "Discover Panama: The Land Between the Seas" : "ColletteDP",
    "Experience Colombia" : "ColletteEC",
    "Grand Hawaiian Adventure Four Islands featuring Oahu, Big Island, Maui and Kauai" : "ColletteGHA",
    "Hawaiian Adventure Three Islands featuring Oahu, Big Island and Maui" : "ColletteHA",
    "Hawaiian Adventure  Three Islands featuring Oahu, Kauai and Maui" : "ColletteHA",
    "Heritage of America" : "ColletteHOA",
    "Heritage of America featuring the Virginia International Tattoo" : "ColletteHOAVT",
    "Highlights of South America featuring Buenos Aires, Iguazu Falls & Rio de Janeiro" : "ColletteHSA",
    "Highlights of South America featuring the Andean Lakes Crossing, Buenos Aires & Rio de Janeiro" : "ColletteHSAAL",
    "Islands of New England" : "ColletteINE",
    "Journey Through South America featuring Santiago, Andean Lakes Crossing & Rio de Janeiro" : "ColletteJTSA",
    "Journey Through South America with Peru featuring Machu Picchu, Andean Lakes Crossing & Rio de Janeiro" : "ColletteJTSAP",
    "Machu Picchu & Galapagos Wonders featuring a 4-Night Cruise" : "ColletteMPGW",
    "Machu Picchu & the Galapagos Islands" : "ColletteMPGI",
    "Machu Picchu & the Galapagos Islands featuring a 3-Night Cruise & 1-Night Island Stay" : "ColletteMPGICI",
    "Mackinac Island" : "ColletteMI",
    "Mackinac Island featuring the Grand Hotel & Chicago" : "ColletteMI",
    "Mackinac Island featuring the Grand Hotel and the Tulip Time Festival" : "ColletteMITF",
    "Maritimes Coastal Wonders featuring the Cabot Trail" : "ColletteMCW",
    "Maritimes Coastal Wonders  featuring the Royal Nova Scotia International Tattoo" : "ColletteMCWT",
    "Montana: Exploring Big Sky Country featuring Yellowstone and Glacier National Parks" : "ColletteMBS",
    "Nashville & the Smoky Mountains Holiday featuring Gatlinburg & Asheville" : "ColletteNSMH",
    "National Parks of America" : "ColletteNPA",
    "Pacific Northwest & California featuring Washington, Oregon and California" : "CollettePNWC",
    "Painted Canyons of the West featuring Utah’s Five National Parks" : "CollettePCW",
    "Patagonia: Edge of the World featuring Argentina, Chile, and a 4-Night Patagonia Cruise" : "CollettePEW",
    "Peru: Ancient Land of Mysteries featuring Puno" : "CollettePALM",
    "Peru: From Lima to Lake Titicaca" : "CollettePLLT",
    "Peru: From Lima to the Sacred Valley" : "CollettePLSV",
    "Peru: Machu Picchu and Lake Titicaca" : "CollettePMPLT",
    "Roaming Coastal Maine featuring Portland, Acadia & Penobscot Bay" : "ColletteRCM",
    "Southern Charm featuring Charleston, Savannah & Jekyll Island" : "ColletteSC",
    "Southern Charm featuring Charleston, Jekyll Island & Savannah" : "ColletteSC",
    "Southern Charm featuring Jekyll Island, Savannah & Charleston" : "ColletteSC",
    "Southern Charm Holiday" : "ColletteSCH",
    "Spotlight on Boston" : "ColletteSB",
    "Spotlight on Montreal" : "ColletteSM",
    "Spotlight on Nashville" : "ColletteSN",
    "Spotlight on New Orleans" : "ColletteSNO",
    "Spotlight on New Orleans featuring Carnival" : "ColletteNOC",
    "Spotlight on New Orleans featuring Jazz Fest" : "ColletteSNOJF",
    "Spotlight on New Orleans Holiday" : "ColletteSNOH",
    "Spotlight on New York City" : "ColletteSNYC",
    "Spotlight on New York City Holiday" : "ColletteSNYCH",
    "Spotlight on San Antonio" : "ColletteSSA",
    "Spotlight on San Antonio featuring the San Antonio Stock Show & Rodeo" : "ColletteSSASSR",
    "Spotlight on San Antonio Holiday" : "ColletteSSAH",
    "Spotlight on Santa Fe" : "ColletteSSF",
    "Spotlight on Santa Fe Holiday" : "ColletteSSFH",
    "Spotlight on Savannah" : "ColletteSS",
    "Spotlight on South Dakota featuring Mount Rushmore & The Badlands" : "ColletteSSD",
    "Spotlight on Washington, D.C. Exploring America's Capital" : "ColletteSWDC",
    "The Best of Eastern Canada featuring Montreal, Quebec City, Ottawa, Niagara Falls & Toronto" : "ColletteBEC",
    "The Best of Eastern Canada featuring Niagara Falls, Ottawa, Quebec City & Montreal" : "ColletteBEC",
    "The Colorado Rockies featuring National Parks and Historic Trains" : "ColletteCRNP",
    "The Complete South America featuring Peru & Machu Picchu" : "ColletteCSA",
    "Tropical Costa Rica" : "ColletteTCR",
    "Washington, D.C., Niagara Falls & New York City" : "ColletteWDCNFNYC",
    "Western Canada’s Rockies, Lakes & Wine Country" : "ColletteWCR",
    "Western Canada’s Rockies, Lakes & Wine Country featuring the Calgary Stampede" : "ColletteWCRS",
    "Winter in Yellowstone" : "ColletteWY",
    "Wonders of Newfoundland featuring Lighthouses, Iceburg Alley, & Gros Morne" : "ColletteWNF"
}

## Look up OpCode

In [ ]:
orig_df = pd.read_csv(current_raw_file, encoding='cp1252')
orig_df = orig_df[orig_df['DepartureID'] != 'DepartureID']
OpCode = orig_df['Trip Name'].apply(lambda x : collette_trips[x]) + orig_df['DepartureID'].str[-3:-1]
orig_df['DepartureID'] = OpCode + "-" + orig_df['DepartureID'].str.split(pat='-', expand=True)[1]

orig_df

## Build Current Data

In [ ]:
curr_df = orig_df.drop_duplicates(subset=['DepartureID', 'field']).pivot(index='DepartureID', columns='field', values='value').reset_index()
curr_df = curr_df[['DepartureID', 'DepartureDate', 'ActualPriceUSD', 'OriginalPriceUSD', 'ActualPriceAUD', 'OriginalPriceAUD', 'Available', 'Type', 'Status', 'Notes']]

curr_df['DepartureDate'] = pd.to_datetime(curr_df['DepartureDate'], format='%d-%b-%Y')
curr_df.sort_values(by=['DepartureDate'], ascending=True, inplace=True)
curr_df = curr_df.loc[curr_df['DepartureDate'] < datetime(year=2023, month=1, day=1, hour=0, minute=0)]

report_ID = curr_df['DepartureDate'].apply(lambda x : None if x < datetime(year=2022, month=1, day=1, hour=0, minute=0) else report_ID_2022)
curr_df.insert(0, 'ReportID', report_ID)

op_code = curr_df['DepartureID'].str.split(pat='-', expand=True)[0]

curr_df.insert(11, 'Pax', np.NaN)

curr_df['ActualPriceUSD'] = curr_df['ActualPriceUSD'].str.replace('$', '').astype(float)
curr_df['OriginalPriceUSD'] = curr_df['OriginalPriceUSD'].str.replace('$', '').astype(float)
curr_df['ActualPriceAUD'] = curr_df['ActualPriceAUD'].str.replace('$', '').astype(float)
curr_df['OriginalPriceAUD'] = curr_df['OriginalPriceAUD'].str.replace('$', '').astype(float)

booleanDictionary = {'True': True, 'False': False}
curr_df['Available'] = curr_df['Available'].replace(booleanDictionary)

curr_df

## Build Previous Data

In [ ]:
prev_df = pd.read_csv(previous_report_file)

prev_df['ReportID'] = prev_df['ReportID'].astype(int)

departure_code = prev_df['DepartureID'].str.split(pat='-', expand=True)[1]
day_numbers = departure_code.str[0:2]
get_char = lambda x : str(ord(x[2]) - 64)
month_numbers = departure_code.apply(get_char)
year_numbers = departure_code.str[3:5]
departure_date = pd.to_datetime(day_numbers + '-' + month_numbers + '-' + year_numbers, format='%d-%m-%y')

prev_df.insert(2, 'DepartureDate', departure_date)

prev_df = prev_df.loc[prev_df['DepartureDate'] >= datetime(year=2022, month=1, day=1, hour=0, minute=0)]

prev_df.sort_values(by=['DepartureDate'], ascending=True, inplace=True)

prev_df['Available'] = prev_df['Available'].astype(bool)
prev_df['Notes'] = prev_df['Notes'].astype(str)
prev_df['Notes'] = prev_df['Notes'].str.replace('nan', '')

prev_df

## Check Departure/Status Count (2022)

In [ ]:
curr_plot_df = curr_df['DepartureID'].loc[curr_df['DepartureDate'].dt.year == 2022].groupby([curr_df['DepartureDate'].dt.month, curr_df['Status']]).count()
prev_plot_df = prev_df['DepartureID'].loc[prev_df['DepartureDate'].dt.year == 2022].groupby([prev_df['DepartureDate'].dt.month, prev_df['Status']]).count()
plot_df = pd.concat([prev_plot_df, curr_plot_df], axis=1)
plot_df.columns = ['Previous', 'Current']
plot_df = plot_df.unstack(level=-1, fill_value=0)

months = plot_df.index
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


if 'Available' in plot_df['Previous'].columns:
    previous_available_counts = plot_df.loc[:, ('Previous', 'Available')]
    previous_available_counts.loc[previous_available_counts.isna()] = 0
else:
    previous_available_counts = 0

if 'Limited' in plot_df['Previous'].columns:
    previous_limited_counts = plot_df.loc[:, ('Previous', 'Limited')]
    previous_limited_counts.loc[previous_limited_counts.isna()] = 0
else:
    previous_limited_counts = 0

if 'Sold Out' in plot_df['Previous'].columns:
    previous_soldout_counts = plot_df.loc[:, ('Previous', 'Sold Out')]
    previous_soldout_counts.loc[previous_soldout_counts.isna()] = 0
else:
    previous_soldout_counts = 0

if 'Cancelled' in plot_df['Previous'].columns:
    previous_cancelled_counts = plot_df.loc[:, ('Previous', 'Cancelled')]
    previous_cancelled_counts.loc[previous_cancelled_counts.isna()] = 0
else:
    previous_cancelled_counts = 0

if 'Cancelled / Sold Out' in plot_df['Previous'].columns:
    previous_removed_counts = plot_df.loc[:, ('Previous', 'Cancelled / Sold Out')]
    previous_removed_counts.loc[previous_removed_counts.isna()] = 0
else:
    previous_removed_counts = 0


if 'Available' in plot_df['Current'].columns:
    current_available_counts = plot_df.loc[:, ('Current', 'Available')]
    current_available_counts.loc[current_available_counts.isna()] = 0
else:
    current_available_counts = 0

if 'Limited' in plot_df['Current'].columns:
    current_limited_counts = plot_df.loc[:, ('Current', 'Limited')]
    current_limited_counts.loc[current_limited_counts.isna()] = 0
else:
    current_limited_counts = 0

if 'Sold Out' in plot_df['Current'].columns:
    current_soldout_counts = plot_df.loc[:, ('Current', 'Sold Out')]
    current_soldout_counts.loc[current_soldout_counts.isna()] = 0
else:
    current_soldout_counts = 0

if 'Cancelled' in plot_df['Current'].columns:
    current_cancelled_counts = plot_df.loc[:, ('Current', 'Cancelled')]
    current_cancelled_counts.loc[current_cancelled_counts.isna()] = 0
else:
    current_cancelled_counts = 0

if 'Cancelled / Sold Out' in plot_df['Current'].columns:
    current_removed_counts = plot_df.loc[:, ('Current', 'Cancelled / Sold Out')]
    current_removed_counts.loc[current_cancelled_counts.isna()] = 0
else:
    current_removed_counts = 0


plt.style.use('fivethirtyeight')
width = 0.35
fig, ax = plt.subplots()

ax.bar(months - width/2, previous_available_counts, width=width, color='#1666BE', label='Available')
ax.bar(months - width/2, previous_limited_counts, width=width, color='#418C5C', bottom=previous_available_counts, label='Limited')
ax.bar(months - width/2, previous_soldout_counts, width=width, color='#CE8D48', bottom=previous_available_counts + previous_limited_counts, label='Sold Out')
ax.bar(months - width/2, previous_cancelled_counts, width=width, color='#A4A49C', bottom=previous_available_counts + previous_limited_counts + previous_soldout_counts, label='Cancelled')
ax.bar(months - width/2, previous_removed_counts, width=width, color='#AD1E45', bottom=previous_available_counts + previous_limited_counts + previous_soldout_counts + previous_cancelled_counts, label='Removed from Website')

ax.bar(months + width/2, current_available_counts, width=width, color='#1666BE')
ax.bar(months + width/2, current_limited_counts, width=width, color='#418C5C', bottom=current_available_counts)
ax.bar(months + width/2, current_soldout_counts, width=width, color='#CE8D48', bottom=current_available_counts + current_limited_counts)
ax.bar(months + width/2, current_cancelled_counts, width=width, color='#A4A49C', bottom=current_available_counts + current_limited_counts + current_soldout_counts)

plt.xticks(ticks=months, labels=month_labels)
plt.legend()
plt.show()

## Check Departure/Status Changes

In [ ]:
curr_status_df = curr_df[['DepartureID', 'DepartureDate', 'Status']].set_index(['DepartureID', 'DepartureDate'])
prev_status_df = prev_df[['DepartureID', 'DepartureDate', 'Status']].set_index(['DepartureID', 'DepartureDate'])
status_df = pd.concat([prev_status_df, curr_status_df], axis=1).reset_index('DepartureDate').sort_values(by='DepartureDate')
status_df.columns = ['DepartureDate', 'Previous', 'Current']

### New Departures

In [ ]:
new_departures = status_df.loc[status_df['Previous'].isna()]

new_departures

### Export New Departures

In [ ]:
if new_departures.empty:
    print('No new departures...')

else:
    export_departures = new_departures['DepartureDate'].reset_index()
    split_id = export_departures['DepartureID'].str.split("-", expand=True)
    export_departures.insert(1, 'OpCode', split_id[0])
    export_departures.insert(2, 'DepartureCode', split_id[1])
    export_departures.insert(4, 'Season', export_departures['DepartureDate'].dt.year)
    export_departures['DepartureDate'] = export_departures['DepartureDate'].dt.strftime('%d-%b-%Y')
    export_departures.set_index('DepartureID', inplace=True)

    export_departures.to_csv('Reports/collette_new_departures.csv')

### Removed Departures

In [ ]:
removed_departures = status_df.loc[(status_df['Current'].isna()) & (status_df['DepartureDate'] > today)]

removed_departures

### Status Changes

In [ ]:
changed_departures = status_df.loc[(status_df['Previous'] != status_df['Current']) & (status_df['Current'].notna())]

changed_departures

## Filter Previous-Past Departures

### Departure date < Today

In [ ]:
prev_past_df = prev_df[prev_df['DepartureDate'] <= today].sort_values(by=['DepartureDate'], ascending=True)

prev_past_df

## Filter Previous-Future Departures

### Departure date > Today
#### If previously 'Available' --> Status = 'Cancelled / Sold Out'

In [ ]:
prev_future_df = prev_df.loc[prev_df['DepartureDate'] > today].sort_values(by=['DepartureDate'], ascending=True)
filt = (prev_future_df['Available'] == True)
prev_future_df.loc[filt, ['Available', 'Status', 'Notes']] = [False, 'Cancelled / Sold Out', 'Removed from website']

prev_future_df

## Combine Current and Previous-Past Data

In [ ]:
new_df = pd.concat([curr_df, prev_past_df]).sort_values(by=['DepartureDate'], ascending=True)

new_df

## Combine Current and Previous-Future Data

In [ ]:
new_df = pd.concat([new_df, prev_future_df]).sort_values(by=['ReportID', 'DepartureID'], ascending=True).drop_duplicates(subset='DepartureID', keep='last').sort_values(by='DepartureDate', ascending=True)

new_df

## Check Mixed Data

In [ ]:
# prof = pandas_profiling.ProfileReport(new_df)
# prof.to_file(output_file='gate1_report.html')

## Export CSV

In [ ]:
new_df.drop(columns='ReportID', inplace=True)

report_ID = new_df['DepartureDate'].apply(lambda x : None if x < datetime(year=2022, month=1, day=1, hour=0, minute=0) else report_ID_2022)
new_df.insert(0, 'ReportID', report_ID)

new_df.drop(columns='DepartureDate', inplace=True)
new_df.set_index(['ReportID', 'DepartureID'], verify_integrity=True, inplace=True)

new_df.to_csv(new_report_file)